In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from financial.momentum.models.kerasAdvanced import KerasAdvancedModelFactory2
from financial.momentum.experiment.modelExperiment import ModelExperimentFactory

2025-07-04 13:39:38.662949: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 13:39:38.926209: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-04 13:39:42.332193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv
import financial.data as fd
from financial.io.file.cache import FileCache
import pandas as pd

In [6]:
load_dotenv(dotenv_path=find_dotenv())

cache = os.environ["CACHE"]
model = os.environ["MODEL"]
print(model)
print(os.environ["DATA"])
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=cache+"/"))
print(datastore) 

/home/manidmt/Universidad/TFG/OTRI/models/keras
/home/manidmt/Universidad/TFG/OTRI/data
CachedDataStore with 946 data sources [cache stats: {'size': 0, 'hit': 0, 'miss': 0, 'write': 0, 'read': 0, 'update': 0}]


In [7]:
factory = KerasAdvancedModelFactory2()

## Single ticker

In [8]:
config = {
    "mode": "global",  
    "datastore": datastore,
    "ticker": "^GSPC",
    "model_factory": factory,
    "name": "lstm_test_gspc_1",
    "start_year": "2000-01-01",
    "end_year": "2023-12-31",
    "lookahead": 20,
    "horizon": 90,
    "model_params": {
        "architecture": "lstm",  # <--- lo importante aquí
        "topology": {
            "layers": [32,16],
            "activation": {
                "hidden": "tanh",
                "output": "linear"
            }
        },
        "optimization": {
            "optimizer": "adam",
            "loss": "mean_squared_error",
            "metrics": ["mae"],
            "epochs": 20,  # para pruebas iniciales
            "batch_size": 32,
            "validation_split": 0.1
        }
    }
}

In [14]:
experiment = ModelExperimentFactory.create_experiment(config)
experiment.run()
print(experiment.predictions)

lstm architecture selected for model lstm_test_gspc_1.2016-01-01
→ Entrenando modelo lstm_test_gspc_1.2016-01-01 con 3935 muestras
Architecture: mlp
Fitting model lstm_test_gspc_1.2016-01-01 with architecture mlp
Epoch 1/300
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0000e+00 - loss: 0.0541
Epoch 2/300
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0000e+00 - loss: 0.0042
Epoch 3/300
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0000e+00 - loss: 0.0025
Epoch 4/300
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0000e+00 - loss: 0.0024
Epoch 5/300
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0000e+00 - loss: 0.0025
Epoch 6/300
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0000e+00 - loss: 0.0025
Epoch 7/300
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0000e+00 - loss: 0.0024
Epoch 8/300
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0000e+00 - loss: 0.0026
Epoch 9/300
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0

KeyboardInterrupt: 

In [ ]:
absolute_predictions = experiment.reconstruct_absolute_predictions_from_relative()
print(absolute_predictions)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style

# Adjusting the size of matplotlib
import matplotlib as mpl

mpl.rc('figure', figsize=(16, 9))
mpl.__version__
style.use('ggplot')
def plot_series(series: list[pd.Series], start_date: str=None, end_date: str=None):
    for i, serie in enumerate(series):
            linewidth = 1.0  # Grosor de línea predeterminado
            if i == 0:  # Si es la primera serie en la lista (índice 0)
                linewidth = 3.0  # Grosor de línea mayor
            serie[start_date:end_date].plot(linewidth=linewidth)

In [ ]:
data = datastore.get_data("^GSPC", "2000-01-01", "2023-12-31")

In [ ]:
plot_series([data, absolute_predictions])

In [ ]:
plot_series([data, absolute_predictions], "2020-01-01", "2020-12-31")

In [ ]:
from financial.model import Model
model_r = model_svr = Model.from_file('lstm_test_gspc', path = os.environ["MODEL"])
predictions_r = model_r.get_data(datastore, start_index="2000-01-01", end_index="2023-12-31")
print(predictions_r)

## Multiple tickers

In [ ]:
config = {
    "mode": "global",
    "datastore": datastore,
    "ticker": ["^GSPC", "^TNX"],  # múltiples tickers
    "model_factory": factory,
    "name": "lstm_test_multiticker",
    "start_year": "2000-01-01",
    "end_year": "2023-12-31",
    "lookahead": 20,
    "horizon": 90,
    "model_params": {
        "architecture": "lstm",
        "topology": {
            "layers": [32, 16],
            "activation": {
                "hidden": "tanh",
                "output": "linear"
            }
        },
        "optimization": {
            "optimizer": "adam",
            "loss": "mean_squared_error",
            "metrics": ["mae"],
            "epochs": 20,
            "batch_size": 32,
            "validation_split": 0.1
        }
    }
}

In [ ]:
experiment = ModelExperimentFactory.create_experiment(config)
experiment.run()
print(experiment.predictions)

In [ ]:
absolute_predictions = experiment.reconstruct_absolute_predictions_from_relative()
print(absolute_predictions)

In [ ]:
plot_series([data, absolute_predictions])

In [ ]:
plot_series([data, absolute_predictions], "2020-01-01", "2020-12-31")

## Metrics
SVR: 8455 MSE=712.6578 RMSE=26.6957 MAE=15.6174 MAPE=0.0096 R² = 0.9994 (**1990 - 2024**)

    TRAIN: n=62696 MSE=0.0002 RMSE=0.0134 MAE=0.0092 MAPE=0.6531
    
    TEST: n=13579 MSE=0.0032 RMSE=0.0563 MAE=0.0392 MAPE=2.4412

In [ ]:
target = data[90 + 20:]
print(target.size)
from financial.lab.experiment import Experiment
from sklearn.metrics import r2_score

In [ ]:
def reconstruct(predictions):
    reconstructed_final = data / (1 + predictions)
    return reconstructed_final.shift(20).dropna()

In [ ]:
absolute_predictions_r = reconstruct(predictions_r)
metrics_single = Experiment("lstm_test_gspc", absolute_predictions_r, target)
r2 = r2_score(target, absolute_predictions_r)

print("GLOBAL:")
print(f"n={metrics_single.samples()} MSE={metrics_single.MSE():.4f} RMSE={metrics_single.RMSE():.4f} MAE={metrics_single.MAE():.4f} MAPE={metrics_single.MAPE():.4f} R² = {r2:.4f}")

In [ ]:
metrics_multiple = Experiment("lstm_test_gspc_multiple", absolute_predictions, target)
r2 = r2_score(target, absolute_predictions)

print("GLOBAL:")
print(f"n={metrics_multiple.samples()} MSE={metrics_multiple.MSE():.4f} RMSE={metrics_multiple.RMSE():.4f} MAE={metrics_multiple.MAE():.4f} MAPE={metrics_multiple.MAPE():.4f} R² = {r2:.4f}")

print("TRAIN: ")
results = experiment.train
print(f"n={results.samples()} MSE={results.MSE():.4f} RMSE={results.RMSE():.4f} MAE={results.MAE():.4f} MAPE={results.MAPE():.4f}")

print("TEST: ")
results = experiment.test
print(f"n={results.samples()} MSE={results.MSE():.4f} RMSE={results.RMSE():.4f} MAE={results.MAE():.4f} MAPE={results.MAPE():.4f}")